In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:

# Define paths to your dataset
train_dir = '/Users/mzitoh/Downloads/archive/train'
test_dir = '/Users/mzitoh/Downloads/archive/val'

In [3]:

# Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

In [4]:

# Use ImageDataGenerator to preprocess the images and create batches of data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 13642 images belonging to 14 classes.
Found 98 images belonging to 14 classes.


In [5]:
class_names = train_generator.class_indices.keys()
num_classes = len(class_names)

pd.DataFrame(class_names, columns=['Flower Type'])

,Flower Type
0,astilbe
1,bellflower
2,black_eyed_susan
3,calendula
4,california_poppy
5,carnation
6,common_daisy
7,coreopsis
8,dandelion
9,iris


In [6]:
# plt.figure(figsize=(15, 20))
# for images, labels in train_generator.take(1):
#     for i in range(8):
#         ax = plt.subplot(8, 4, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(cl_nm[labels[i]])
#         plt.axis("off")

In [7]:
# Define the input shape
input_shape = (img_height, img_width, 3)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape),
    # tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

/Users/mzitoh/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 14)             │         1,806 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,170,766 (42.61 MB)

 Trainable params: 11,170,766 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
# model.fit(train_generator, epochs=1)
model.fit(train_generator, epochs=10)

/Users/mzitoh/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


427/427 ━━━━━━━━━━━━━━━━━━━━ 248s 574ms/step - accuracy: 0.3444 - loss: 1.9106


In [10]:
# Evaluate the model
model.evaluate(test_generator)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.5620 - loss: 1.2760


[1.2279479503631592, 0.5612244606018066]

In [11]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/kd/9h3yrkps2vj0t9zs17vt349c0000gn/T/tmpek6lbyw1/assets


INFO:tensorflow:Assets written to: /var/folders/kd/9h3yrkps2vj0t9zs17vt349c0000gn/T/tmpek6lbyw1/assets


: 

In [ ]:
# Save the TensorFlow Lite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)